In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt, rcParams, style
style.use('seaborn-darkgrid')
import seaborn as sns
sns.set_style('darkgrid')
from plotly import express as px, graph_objects as go

from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from statsmodels.graphics.tsaplots import plot_pacf

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.metrics import mean_squared_log_error as msle

import gc
gc.enable()
from warnings import filterwarnings, simplefilter
filterwarnings('ignore')
simplefilter('ignore')

# Holidays

In [ ]:
holidays = pd.read_csv("../input/store-sales-time-series-forecasting/holidays_events.csv")

In [ ]:
holidays.head()

In [ ]:
holidays.isnull().sum()

In [ ]:
holidays.info()

In [ ]:
x_df = holidays['transferred'].value_counts()
x_df.plot.pie(explode = [0,0.1], autopct = '%1.1f%%', shadow = False)
plt.title("Distribution of transferred column")
plt.ylabel('')
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize = (18, 5))
sns.countplot(ax = axes[0], data = holidays, x = 'type', hue = 'transferred')
axes[0].set_title('Transferred Holidays')
sns.countplot(ax = axes[1], data = holidays, x = 'locale', hue = 'transferred')
axes[1].set_title('Transferred Holidays')


# Train data

In [ ]:
train = pd.read_csv("../input/store-sales-time-series-forecasting/train.csv")

In [ ]:
train.head()

In [ ]:
train.isnull().sum()

In [ ]:
train.info()

In [ ]:
train.shape

In [ ]:
count_fam = train[['sales', 'family']].groupby('family').sum().reset_index()

In [ ]:
count_fam['percent'] = (count_fam['sales'] / train['sales'].sum()) * 100

In [ ]:
count_fam['percent'] = count_fam['percent'].sort_values(ascending = False).head()

In [ ]:
fig = px.pie(count_fam, names='family', values='percent', title = "5 categories that make the most sales", 
            color_discrete_sequence=px.colors.sequential.RdBu)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.update_layout(showlegend=False, title_x=0.5)
# fig.show("png")

In [ ]:
train['year'] = pd.to_datetime(train['date']).dt.year
train['month'] = pd.to_datetime(train['date']).dt.strftime("%B")
train['day'] = pd.to_datetime(train['date']).dt.day_name()

In [ ]:
train.head()

In [ ]:
train['date_str'] = train['date'].astype(str)
train['year_month'] = train['date_str'].str.slice(0,7)

In [ ]:
train_aux = train[['year_month', 'sales', 'onpromotion']].groupby('year_month').mean()
train_aux = train_aux.reset_index()
fig = px.line(train_aux, x = 'year_month', y = 'sales', title = "Average sales over time")
fig.update_layout(title_x=0.5)
fig.show()

In [ ]:
sales_year = train[['date', 'sales', 'onpromotion', 'year_month']].groupby('date').mean()
sales_year = sales_year.reset_index()
sales_year[sales_year['onpromotion'] > 0]

In [ ]:
df = sales_year[sales_year['onpromotion'] > 0]
fig = px.scatter(df, x = "date", y = "sales", color="onpromotion", size_max=25,
                 title = "Total average sales for items on promotion", color_continuous_scale = px.colors.sequential.Viridis)
fig.update_layout(title_x = 0.5)
fig.show()

We see that there is a positive correlation between onpromotion and sales units sold. Thus, when more items of the supermarkets are on promotion, it's more likely to sell them. Also, for more recent years, more items are on promotion and the sales are higher too. There were more items on promotion from family of product for recent years compared to that in 2014.

In [ ]:
sales_year = train.groupby('year').mean()[['sales']]
sales_year = sales_year.reset_index()
sales_month = train.groupby('month').mean()[['sales']]
sales_month = sales_month.reset_index()
sales_day = train.groupby('day').mean()[['sales']]
sales_day = sales_day.reset_index()

fig, axes = plt.subplots(1, 3, figsize = (20, 5))
for ax in fig.axes:
    ax.tick_params(labelrotation=45)
sns.barplot(x=sales_year['year'], y=sales_year['sales'], ax = axes[0], palette = "crest")
axes[0].set_title('Avg Sales by Year')
sns.barplot(x=sales_month['month'], y=sales_month['sales'], ax = axes[1], palette = "rocket")
axes[1].set_title('Avg Sales by Month')
sns.barplot(x=sales_day['day'], y=sales_day['sales'], ax = axes[2], palette = "Blues_d")
axes[2].set_title('Avg Sales by Day of the week')

We notice an increasing trend in sales every year which can either be due to the decrease in oil prices or with growing econonmic conditions of Ecuador. Also, more average units were sold in December which can be due to Christmas.

Taking a closer into the days of the week, we notice that during the weekends customers purchase more compared items than during week days.

In [ ]:
holidays = holidays[(holidays['date'] >= "2013-01-01") & (holidays['date'] <= "2017-08-15")]

In [ ]:
holidays.head()

In [ ]:
holidays.shape

# Oil

In [ ]:
oil = pd.read_csv("../input/store-sales-time-series-forecasting/oil.csv")

In [ ]:
oil.head()

In [ ]:
oil.shape

In [ ]:
oil['date'] = pd.to_datetime(oil['date'])

In [ ]:
oil = oil[(oil['date'] >= "2013-01-01") & (oil['date'] <= "2017-08-15")]

In [ ]:
oil.shape

Replacing NaN by taking a mean of all values for that month

In [ ]:
oil['year'] = pd.to_datetime(oil['date']).dt.year
oil['month'] = pd.to_datetime(oil['date']).dt.strftime("%B")
oil['day'] = pd.to_datetime(oil['date']).dt.day_name()

In [ ]:
oil.head()

In [ ]:
oil.isnull().sum()

In [ ]:
mean_jan_2013 = oil[(oil['year'] == 2013) & (oil['month'] == "January")]['dcoilwtico'].mean(skipna = True)
mean_feb_2013 = oil[(oil['year'] == 2013) & (oil['month'] == "February")]['dcoilwtico'].mean(skipna = True)
mean_march_2013 = oil[(oil['year'] == 2013) & (oil['month'] == "March")]['dcoilwtico'].mean(skipna = True)
mean_may_2013 = oil[(oil['year'] == 2013) & (oil['month'] == "May")]['dcoilwtico'].mean(skipna = True)
mean_july_2013 = oil[(oil['year'] == 2013) & (oil['month'] == "July")]['dcoilwtico'].mean(skipna = True)
mean_sept_2013 = oil[(oil['year'] == 2013) & (oil['month'] == "September")]['dcoilwtico'].mean(skipna = True)
mean_nov_2013 = oil[(oil['year'] == 2013) & (oil['month'] == "November")]['dcoilwtico'].mean(skipna = True)
mean_dec_2013 = oil[(oil['year'] == 2013) & (oil['month'] == "December")]['dcoilwtico'].mean(skipna = True)

In [ ]:
oil.loc[(oil['year'] == 2013) & (oil['month'] == "January"), "dcoilwtico"] = mean_jan_2013
oil.loc[(oil['year'] == 2013) & (oil['month'] == "February"), "dcoilwtico"] = mean_feb_2013
oil.loc[(oil['year'] == 2013) & (oil['month'] == "March"), "dcoilwtico"] = mean_march_2013
oil.loc[(oil['year'] == 2013) & (oil['month'] == "May"), "dcoilwtico"] = mean_may_2013
oil.loc[(oil['year'] == 2013) & (oil['month'] == "July"), "dcoilwtico"] = mean_july_2013
oil.loc[(oil['year'] == 2013) & (oil['month'] == "September"), "dcoilwtico"] = mean_sept_2013
oil.loc[(oil['year'] == 2013) & (oil['month'] == "November"), "dcoilwtico"] = mean_nov_2013
oil.loc[(oil['year'] == 2013) & (oil['month'] == "December"), "dcoilwtico"] = mean_dec_2013

In [ ]:
mean_july_2017 = oil[(oil['year'] == 2017) & (oil['month'] == "July")]['dcoilwtico'].mean(skipna = True)
mean_may_2017 = oil[(oil['year'] == 2017) & (oil['month'] == "May")]['dcoilwtico'].mean(skipna = True)
mean_april_2017 = oil[(oil['year'] == 2017) & (oil['month'] == "April")]['dcoilwtico'].mean(skipna = True)
mean_feb_2017 = oil[(oil['year'] == 2017) & (oil['month'] == "February")]['dcoilwtico'].mean(skipna = True)
mean_jan_2017 = oil[(oil['year'] == 2017) & (oil['month'] == "January")]['dcoilwtico'].mean(skipna = True)

In [ ]:
oil.loc[(oil['year'] == 2017) & (oil['month'] == "January"), "dcoilwtico"] = mean_jan_2017
oil.loc[(oil['year'] == 2017) & (oil['month'] == "February"), "dcoilwtico"] = mean_feb_2017
oil.loc[(oil['year'] == 2017) & (oil['month'] == "April"), "dcoilwtico"] = mean_april_2017
oil.loc[(oil['year'] == 2017) & (oil['month'] == "May"), "dcoilwtico"] = mean_may_2017
oil.loc[(oil['year'] == 2017) & (oil['month'] == "July"), "dcoilwtico"] = mean_july_2017

In [ ]:
mean_jan_2014 = oil[(oil['year'] == 2014) & (oil['month'] == "January")]['dcoilwtico'].mean(skipna = True)
mean_feb_2014 = oil[(oil['year'] == 2014) & (oil['month'] == "February")]['dcoilwtico'].mean(skipna = True)
mean_april_2014 = oil[(oil['year'] == 2014) & (oil['month'] == "April")]['dcoilwtico'].mean(skipna = True)
mean_may_2014 = oil[(oil['year'] == 2014) & (oil['month'] == "May")]['dcoilwtico'].mean(skipna = True)
mean_july_2014 = oil[(oil['year'] == 2014) & (oil['month'] == "July")]['dcoilwtico'].mean(skipna = True)
mean_sept_2014 = oil[(oil['year'] == 2014) & (oil['month'] == "September")]['dcoilwtico'].mean(skipna = True)
mean_nov_2014 = oil[(oil['year'] == 2014) & (oil['month'] == "November")]['dcoilwtico'].mean(skipna = True)
mean_dec_2014 = oil[(oil['year'] == 2014) & (oil['month'] == "December")]['dcoilwtico'].mean(skipna = True)

In [ ]:
oil.loc[(oil['year'] == 2014) & (oil['month'] == "January"), "dcoilwtico"] = mean_jan_2014
oil.loc[(oil['year'] == 2014) & (oil['month'] == "February"), "dcoilwtico"] = mean_feb_2014
oil.loc[(oil['year'] == 2014) & (oil['month'] == "April"), "dcoilwtico"] = mean_april_2014
oil.loc[(oil['year'] == 2014) & (oil['month'] == "May"), "dcoilwtico"] = mean_may_2014
oil.loc[(oil['year'] == 2014) & (oil['month'] == "July"), "dcoilwtico"] = mean_july_2014
oil.loc[(oil['year'] == 2014) & (oil['month'] == "September"), "dcoilwtico"] = mean_sept_2014
oil.loc[(oil['year'] == 2014) & (oil['month'] == "November"), "dcoilwtico"] = mean_nov_2014
oil.loc[(oil['year'] == 2014) & (oil['month'] == "December"), "dcoilwtico"] = mean_dec_2014

In [ ]:
mean_jan_2015 = oil[(oil['year'] == 2015) & (oil['month'] == "January")]['dcoilwtico'].mean(skipna = True)
mean_feb_2015 = oil[(oil['year'] == 2015) & (oil['month'] == "February")]['dcoilwtico'].mean(skipna = True)
mean_april_2015 = oil[(oil['year'] == 2015) & (oil['month'] == "April")]['dcoilwtico'].mean(skipna = True)
mean_may_2015 = oil[(oil['year'] == 2015) & (oil['month'] == "May")]['dcoilwtico'].mean(skipna = True)
mean_july_2015 = oil[(oil['year'] == 2015) & (oil['month'] == "July")]['dcoilwtico'].mean(skipna = True)
mean_sept_2015 = oil[(oil['year'] == 2015) & (oil['month'] == "September")]['dcoilwtico'].mean(skipna = True)
mean_nov_2015 = oil[(oil['year'] == 2015) & (oil['month'] == "November")]['dcoilwtico'].mean(skipna = True)
mean_dec_2015 = oil[(oil['year'] == 2015) & (oil['month'] == "December")]['dcoilwtico'].mean(skipna = True)

In [ ]:
oil.loc[(oil['year'] == 2015) & (oil['month'] == "January"), "dcoilwtico"] = mean_jan_2015
oil.loc[(oil['year'] == 2015) & (oil['month'] == "February"), "dcoilwtico"] = mean_feb_2015
oil.loc[(oil['year'] == 2015) & (oil['month'] == "April"), "dcoilwtico"] = mean_april_2015
oil.loc[(oil['year'] == 2015) & (oil['month'] == "May"), "dcoilwtico"] = mean_may_2015
oil.loc[(oil['year'] == 2015) & (oil['month'] == "July"), "dcoilwtico"] = mean_july_2015
oil.loc[(oil['year'] == 2015) & (oil['month'] == "September"), "dcoilwtico"] = mean_sept_2015
oil.loc[(oil['year'] == 2015) & (oil['month'] == "November"), "dcoilwtico"] = mean_nov_2015
oil.loc[(oil['year'] == 2015) & (oil['month'] == "December"), "dcoilwtico"] = mean_dec_2015

In [ ]:
mean_jan_2016 = oil[(oil['year'] == 2016) & (oil['month'] == "January")]['dcoilwtico'].mean(skipna = True)
mean_feb_2016 = oil[(oil['year'] == 2016) & (oil['month'] == "February")]['dcoilwtico'].mean(skipna = True)
mean_march_2016 = oil[(oil['year'] == 2016) & (oil['month'] == "March")]['dcoilwtico'].mean(skipna = True)
mean_may_2016 = oil[(oil['year'] == 2016) & (oil['month'] == "May")]['dcoilwtico'].mean(skipna = True)
mean_july_2016 = oil[(oil['year'] == 2016) & (oil['month'] == "July")]['dcoilwtico'].mean(skipna = True)
mean_sept_2016 = oil[(oil['year'] == 2016) & (oil['month'] == "September")]['dcoilwtico'].mean(skipna = True)
mean_nov_2016 = oil[(oil['year'] == 2016) & (oil['month'] == "November")]['dcoilwtico'].mean(skipna = True)
mean_dec_2016 = oil[(oil['year'] == 2016) & (oil['month'] == "December")]['dcoilwtico'].mean(skipna = True)

In [ ]:
oil.loc[(oil['year'] == 2016) & (oil['month'] == "January"), "dcoilwtico"] = mean_jan_2016
oil.loc[(oil['year'] == 2016) & (oil['month'] == "February"), "dcoilwtico"] = mean_feb_2016
oil.loc[(oil['year'] == 2016) & (oil['month'] == "March"), "dcoilwtico"] = mean_march_2016
oil.loc[(oil['year'] == 2016) & (oil['month'] == "May"), "dcoilwtico"] = mean_may_2016
oil.loc[(oil['year'] == 2016) & (oil['month'] == "July"), "dcoilwtico"] = mean_july_2016
oil.loc[(oil['year'] == 2016) & (oil['month'] == "September"), "dcoilwtico"] = mean_sept_2016
oil.loc[(oil['year'] == 2016) & (oil['month'] == "November"), "dcoilwtico"] = mean_nov_2016
oil.loc[(oil['year'] == 2016) & (oil['month'] == "December"), "dcoilwtico"] = mean_dec_2016

In [ ]:
fig = px.line(oil, x="date", y="dcoilwtico", title='Oil prices over time')
fig.update_layout(title_x = 0.5)
fig.show()

# Stores

In [ ]:
transactions = pd.read_csv("../input/store-sales-time-series-forecasting/transactions.csv")

In [ ]:
stores = pd.read_csv("../input/store-sales-time-series-forecasting/stores.csv")

In [ ]:
stores.head()

In [ ]:
store_trans = pd.merge(stores, transactions, how = 'left', on = 'store_nbr')
store_trans.head()

In [ ]:
store_trans = store_trans.groupby(['store_nbr', 'type']).mean()[['transactions']].reset_index(level = 0).reset_index(level = 0)[['store_nbr', 'type', 
                                                                                                'transactions']]
store_trans = store_trans.sort_values("transactions", ascending = False)

fig = px.bar(store_trans, x = "store_nbr", y = "transactions", color = "type", text = 'type',
            barmode = 'stack', title = "Total number of transactions for each store")
fig.update_layout(title_x = 0.5)
fig.show()

In [ ]:
train_store = pd.merge(train, stores, how = 'left', on = 'store_nbr')

In [ ]:
train_store.head()

In [ ]:
train_store = train_store.groupby(['onpromotion', 'family']).sum()[['sales']].reset_index(level = 0).reset_index(level = 0)[['onpromotion', 'family', 
                                                                                                'sales']]
train_store = train_store.sort_values("sales", ascending = False).head(25)

In [ ]:
train_store.head()

In [ ]:
fig = px.bar(train_store, x = "family", y = "sales", color = "onpromotion", text = 'onpromotion',
            barmode = 'stack', title = "Total number of items sold in each family of products")
fig.update_layout(title_x = 0.5)
fig.show()

# Transactions

In [ ]:
transactions = pd.read_csv("../input/store-sales-time-series-forecasting/transactions.csv")

In [ ]:
transactions.head()

In [ ]:
transactions.shape

In [ ]:
transactions = transactions[(transactions['date'] >= "2013-01-01") & (transactions['date'] <= "2017-08-15")]
transactions.head()

In [ ]:
transactions.isnull().sum()

In [ ]:
transactions['date'] = pd.to_datetime(transactions['date'])
trans = transactions[['date', 'transactions']].groupby('date').mean()
trans = trans.reset_index()

In [ ]:
fig = px.line(trans, x="date", y="transactions", title='Average transactions over time')
fig.update_layout(title_x = 0.5)
fig.show()

We can notice that after July 2015 the number of transaction has been stable.

# Modeling

## Reading the data

In [ ]:
# Train dataset
train = pd.read_csv('../input/store-sales-time-series-forecasting/train.csv', parse_dates = ['date'], 
                    infer_datetime_format = True, dtype = {'store_nbr' : 'category', 'family' : 'category'},
                    usecols = ['date', 'store_nbr', 'family', 'sales'])

train['date'] = train.date.dt.to_period('D')

train = train.set_index(['date', 'store_nbr', 'family']).sort_index()


# Test dataset
test = pd.read_csv('../input/store-sales-time-series-forecasting/test.csv', parse_dates = ['date'], 
                   infer_datetime_format = True)

test['date'] = test.date.dt.to_period('D')

test = test.set_index(['date', 'store_nbr', 'family']).sort_values('id')


# Stores dataset
stores = pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')


# Oil dataset
oil = pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv', parse_dates = ['date'], 
                  infer_datetime_format = True, index_col = 'date').to_period('D')

# Holidays and events dataset
hol = pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv', parse_dates = ['date'], 
                  infer_datetime_format = True, index_col = 'date').to_period('D')

# in order to avoid any false positives we will only be taking 'national' holidays into account
hol = hol[hol.locale == 'National'] 

# Removing duplicated holiday at the same date
hol = hol.groupby(hol.index).first()

## Data Preprocessing

In [ ]:
processed_data = pd.DataFrame(index = pd.date_range('2013-01-01', '2017-08-31')).to_period('D')

processed_data.head()

In [ ]:
oil.head()

In [ ]:
## creating a new column average oil which has values of mean of last 7 entries
oil['avg_oil'] = oil['dcoilwtico'].rolling(7).mean()

In [ ]:
## ading avg oil to preprocessed data
processed_data = processed_data.join(oil.avg_oil)

## replacing nan value using forward fill method.
processed_data['avg_oil'].fillna(method = 'ffill', inplace = True) 
processed_data.dropna(inplace = True)

## creating lag features
processed_data['Lag_1'] = processed_data['avg_oil'].shift(1)
processed_data['Lag_2'] = processed_data['avg_oil'].shift(2)
processed_data['Lag_3'] = processed_data['avg_oil'].shift(3)
processed_data.dropna(inplace = True)

Now let's join the holidays dataset to the dataset obtained above

In [ ]:
processed_data = processed_data.join(hol) 

In [ ]:
## Distinguishing the days of the week as weekday or weekend
processed_data['dayoftheweek'] = processed_data.index.dayofweek
processed_data['weekday'] = 1
processed_data.loc[processed_data.dayoftheweek > 4, 'weekday'] = 0 

## Processing the type of day to either weekday or weekend
processed_data.loc[processed_data.type == 'Work Day', 'weekday'] = 1 
processed_data.loc[processed_data.type == 'Transfer', 'weekday'] = 0 
processed_data.loc[processed_data.type == 'Bridge', 'weekday'] = 0 

## treating the holidays that were transferred on a weekday as holiday
processed_data.loc[(processed_data.type == 'Holiday') & (processed_data.transferred == False), 'weekday'] = 0 
processed_data.loc[(processed_data.type == 'Holiday') & (processed_data.transferred == True), 'weekday'] = 1 

## Encoding the columns 'dayoftheweek' and 'type'
processed_data = pd.get_dummies(processed_data, columns = ['dayoftheweek'], drop_first = True) 
processed_data = pd.get_dummies(processed_data, columns = ['type']) 

## Dropping the columns that not needed anymore
processed_data.drop(['locale', 'locale_name', 'description', 'transferred'], axis = 1, inplace = True)

In [ ]:
processed_data.head()

In [ ]:
## Separating target values
y = train.unstack(['store_nbr', 'family']).loc['2017-04-30':'2017-08-15']

## creating linear trend 
fourier = CalendarFourier(freq = 'W', order = 4)
dp = DeterministicProcess(index = y.index, order = 1, seasonal = False,
                          constant = False, additional_terms = [fourier], drop = True)

## preprocessed training data
x = dp.in_sample()
x = x.join(processed_data)

In [ ]:
x.head()

In [ ]:
y.head()

In [ ]:
x.shape

In [ ]:
# Creating test dataset to predict sales for next 16 days
xtest = dp.out_of_sample(steps = 16) 
xtest = xtest.join(processed_data)

In [ ]:
xtest.head()

In [ ]:
xtest.shape

*italicized text*## Linear Regression Model

In [ ]:
linear_model = LinearRegression(fit_intercept = True, n_jobs = -1, normalize = True)
linear_model.fit(x, y)

yfit_linear = pd.DataFrame(linear_model.predict(x), index = x.index, columns = y.columns).clip(0.)
ypred_linear = pd.DataFrame(linear_model.predict(xtest), index = xtest.index, columns = y.columns).clip(0.)

In [ ]:
## Creating Submision file for CSV
from sklearn.metrics import mean_squared_log_error
y_pred_2 = yfit_linear.stack(['store_nbr', 'family']).clip(0.)
y_ = y.stack(['store_nbr', 'family']).clip(0.)

y_['pred'] = y_pred_2.values
print(y_.groupby('family').apply(lambda r : np.sqrt(np.sqrt(mean_squared_log_error(r['sales'], r['pred'])))))
# Looking at error
print('RMSLE : ', np.sqrt(np.sqrt(msle(y_['sales'], y_['pred']))))


ypred_3 = ypred_linear.stack(['store_nbr', 'family'])

sub = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')
sub['sales'] = ypred_3.values
sub.to_csv('new_lr_submission.csv', index = False) # Submit

## Ridge Regression Model

In [ ]:
ridge = Ridge(fit_intercept=True, solver='auto', alpha=0.75, normalize=True)
ridge.fit(x, y)

yfit_ridge = pd.DataFrame(ridge.predict(x), index = x.index, columns = y.columns).clip(0.)
ypred_ridge = pd.DataFrame(ridge.predict(xtest), index = xtest.index, columns = y.columns).clip(0.)

In [ ]:
## Creating Submision file for CSV
from sklearn.metrics import mean_squared_log_error
y_pred_2 = yfit_ridge.stack(['store_nbr', 'family']).clip(0.)
y_ = y.stack(['store_nbr', 'family']).clip(0.)

y_['pred'] = y_pred_2.values
print(y_.groupby('family').apply(lambda r : np.sqrt(np.sqrt(mean_squared_log_error(r['sales'], r['pred'])))))
# Looking at error
print('RMSLE : ', np.sqrt(np.sqrt(msle(y_['sales'], y_['pred']))))


ypred_3 = ypred_ridge.stack(['store_nbr', 'family'])

sub = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')
sub['sales'] = ypred_3.values
sub.to_csv('new_ridge_submission.csv', index = False) # Submit

## Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf=RandomForestRegressor(n_estimators = 500, n_jobs=-1,max_depth=10)

rf.fit(x, y)

yfit_rf = pd.DataFrame(rf.predict(x), index = x.index, columns = y.columns).clip(0.)
ypred_rf = pd.DataFrame(rf.predict(xtest), index = xtest.index, columns = y.columns).clip(0.)

In [ ]:
## Creating Submision file for CSV
from sklearn.metrics import mean_squared_log_error
y_pred_2 = yfit_rf.stack(['store_nbr', 'family']).clip(0.)
y_ = y.stack(['store_nbr', 'family']).clip(0.)

y_['pred'] = y_pred_2.values
print(y_.groupby('family').apply(lambda r : np.sqrt(np.sqrt(mean_squared_log_error(r['sales'], r['pred'])))))
# Looking at error
print('RMSLE : ', np.sqrt(np.sqrt(msle(y_['sales'], y_['pred']))))


ypred_3 = ypred_rf.stack(['store_nbr', 'family'])

sub = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')
sub['sales'] = ypred_3.values
sub.to_csv('new_rf_submission.csv', index = False) # Submit

In [ ]:
importances = rf.feature_importances_
features = x.columns
indices = np.argsort(importances)
plt.rcParams["figure.figsize"] = (15,8)

plt.title(' Random forest Feature Importances')
plt.barh(range(len(indices)), importances[indices], align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.savefig('RandomforestFeatureImportances.png')
plt.show()

## LASO


In [ ]:
from sklearn.linear_model import Lasso

lasso_model = Lasso(alpha =0.005)
lasso_model.fit(x, y)

yfit_lasso = pd.DataFrame(lasso_model.predict(x), index = x.index, columns = y.columns).clip(0.)
ypred_lasso = pd.DataFrame(lasso_model.predict(xtest), index = xtest.index, columns = y.columns).clip(0.)

In [ ]:
## Creating Submision file for CSV
from sklearn.metrics import mean_squared_log_error
y_pred_2 = yfit_lasso.stack(['store_nbr', 'family']).clip(0.)
y_ = y.stack(['store_nbr', 'family']).clip(0.)

y_['pred'] = y_pred_2.values
print(y_.groupby('family').apply(lambda r : np.sqrt(np.sqrt(mean_squared_log_error(r['sales'], r['pred'])))))
# Looking at error
print('RMSLE : ', np.sqrt(np.sqrt(msle(y_['sales'], y_['pred']))))


ypred_3 = ypred_lasso.stack(['store_nbr', 'family'])

sub = pd.read_csv('sample_submission.csv')
sub['sales'] = ypred_3.values
sub.to_csv('new_lasso_submission.csv', index = False) # Submit

## SVR

In [ ]:
from sklearn.svm import SVR
from joblib import Parallel, delayed
from tqdm.auto import tqdm

class Svr():
    
    def __init__(self, n_jobs=-1, verbose=0):
        
        self.n_jobs = n_jobs
        self.verbose = verbose
        
        self.estimators_ = None
        
    def _estimator_(self, X, y):

        model = SVR(C = 0.2, kernel = 'rbf')
        model.fit(X, y)

        return model

    def fit(self, X, y):
        from tqdm.auto import tqdm
        if self.verbose == 0 :
            self.estimators_ = Parallel(n_jobs=self.n_jobs, 
                                  verbose=0,
                                  )(delayed(self._estimator_)(X, y.iloc[:, i]) for i in range(y.shape[1]))
        else :
            print('Fit Progress')
            self.estimators_ = Parallel(n_jobs=self.n_jobs, 
                                  verbose=0,
                                  )(delayed(self._estimator_)(X, y.iloc[:, i]) for i in tqdm(range(y.shape[1])))
        return
    
    def predict(self, X):
        from tqdm.auto import tqdm
        if self.verbose == 0 :
            y_pred = Parallel(n_jobs=self.n_jobs, 
                              verbose=0)(delayed(e.predict)(X) for e in self.estimators_)
        else :
            print('Predict Progress')
            y_pred = Parallel(n_jobs=self.n_jobs, 
                              verbose=0)(delayed(e.predict)(X) for e in tqdm(self.estimators_))
        
        return np.stack(y_pred, axis=1)

In [ ]:
svr = Svr(n_jobs=-1, verbose=1)
svr.fit(x, y)

yfit_svr = pd.DataFrame(svr.predict(x), index = x.index, columns = y.columns).clip(0.)
ypred_svr = pd.DataFrame(svr.predict(xtest), index = xtest.index, columns = y.columns).clip(0.)

In [ ]:
## Creating Submision file for CSV
from sklearn.metrics import mean_squared_log_error
y_pred_2 = yfit_svr.stack(['store_nbr', 'family']).clip(0.)
y_ = y.stack(['store_nbr', 'family']).clip(0.)

y_['pred'] = y_pred_2.values
print(y_.groupby('family').apply(lambda r : np.sqrt(np.sqrt(mean_squared_log_error(r['sales'], r['pred'])))))
# Looking at error
print('RMSLE : ', np.sqrt(np.sqrt(msle(y_['sales'], y_['pred']))))


ypred_3 = ypred_svr.stack(['store_nbr', 'family'])

sub = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')
sub['sales'] = ypred_3.values
sub.to_csv('new_svr_submission.csv', index = False) # Submit

## XGBoost Regressor

In [ ]:
import xgboost as xgb
xg=xgb.XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 100)
xg.fit(x, y)

yfit_xg = pd.DataFrame(xg.predict(x), index = x.index, columns = y.columns).clip(0.)
ypred_xg = pd.DataFrame(xg.predict(xtest), index = xtest.index, columns = y.columns).clip(0.)

In [ ]:
## Creating Submision file for CSV
from sklearn.metrics import mean_squared_log_error
y_pred_2 = yfit_xg.stack(['store_nbr', 'family']).clip(0.)
y_ = y.stack(['store_nbr', 'family']).clip(0.)

y_['pred'] = y_pred_2.values
print(y_.groupby('family').apply(lambda r : np.sqrt(np.sqrt(mean_squared_log_error(r['sales'], r['pred'])))))
# Looking at error
print('RMSLE : ', np.sqrt(np.sqrt(msle(y_['sales'], y_['pred']))))


ypred_3 = ypred_xg.stack(['store_nbr', 'family'])

sub = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')
sub['sales'] = ypred_3.values
sub.to_csv('new_xg_submission.csv', index = False) # Submit

In [ ]:
feature_important = xg.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
## plot all features
fig=data.nlargest(30, columns="score").plot(kind='bar',title='XGboost Feature importance', figsize = (20,10)).get_figure()
fig.savefig('test.png')


## Blending (time-series technique)

In [ ]:
linear_model = LinearRegression(fit_intercept = True, n_jobs = -1, normalize = True)
linear_model.fit(x, y)

yfit_linear = pd.DataFrame(linear_model.predict(x), index = x.index, columns = y.columns).clip(0.)
ypred_linear = pd.DataFrame(linear_model.predict(xtest), index = xtest.index, columns = y.columns).clip(0.)

In [ ]:
## combining the train and test prediction
y_mean = yfit_linear.append(ypred_linear)


school = y_mean.loc(axis = 1)['sales', :, 'SCHOOL AND OFFICE SUPPLIES']

## adding a lag feature for the school feature
y_mean = y_mean.join(school.shift(1), rsuffix = 'lag1')

x = x.loc['2017-05-01':]

## Concating linear result
x_linear = x.join(y_mean) 
xtest_linear = xtest.join(y_mean)

y_linear = y.loc['2017-05-01':]

### XGB Blending

In [ ]:
xg=xgb.XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 10, alpha = 10, n_estimators = 100)
xg.fit(x, y)

yfit_xg = pd.DataFrame(xg.predict(x), index = x.index, columns = y.columns).clip(0.)
ypred_xg = pd.DataFrame(xg.predict(xtest), index = xtest.index, columns = y.columns).clip(0.)

In [ ]:
## combining the train and test prediction
y_mean = yfit_xg.append(ypred_xg)


school = y_mean.loc(axis = 1)['sales', :, 'SCHOOL AND OFFICE SUPPLIES']

## adding a lag feature for the school feature
y_mean = y_mean.join(school.shift(1), rsuffix = 'lag1')

x = x.loc['2017-05-01':]

## Concating linear result
x_xgb = x.join(y_mean) 
xtest_xgb = xtest.join(y_mean)

y_xbg = y.loc['2017-05-01':]

In [ ]:
x_linear.head()

# Best Model

In [ ]:
from joblib import Parallel, delayed
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.model_selection import TimeSeriesSplit
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingRegressor
import warnings

# SEED for reproducible result
SEED = 42

class ModifiedRegressor():
    
    def __init__(self, n_jobs=-1, verbose=0):
        
        self.n_jobs = n_jobs
        self.verbose = verbose
        
        self.estimators_ = None
        
    def _estimator_(self, X, y):
    
        warnings.simplefilter(action='ignore', category=FutureWarning)
        
        if y.name[2] == 'SCHOOL AND OFFICE SUPPLIES': # Because SCHOOL AND OFFICE SUPPLIES has weird trend, we use decision tree instead.
            r1 = ExtraTreesRegressor(n_estimators = 225, n_jobs=-1,max_depth=10, random_state=SEED)
            r2 = RandomForestRegressor(n_estimators = 225, n_jobs=-1,max_depth=10, random_state=SEED)
            b1 = BaggingRegressor(base_estimator=r1,
                                  n_estimators=10,
                                  n_jobs=-1,
                                  random_state=SEED)
            b2 = BaggingRegressor(base_estimator=r2,
                                  n_estimators=10,
                                  n_jobs=-1,
                                  random_state=SEED)
            model = VotingRegressor([('et', b1), ('rf', b2)]) # Averaging the result
        else:
            ridge = Ridge(fit_intercept=True, solver='auto', alpha=0.75, normalize=True, random_state=SEED)
            svr = SVR(C = 0.2, kernel = 'rbf')
            
            model = VotingRegressor([('ridge', ridge), ('svr', svr)]) # Averaging result
        model.fit(X, y)

        return model

    def fit(self, X, y):
        from tqdm.auto import tqdm
        
        
        if self.verbose == 0 :
            self.estimators_ = Parallel(n_jobs=self.n_jobs, 
                                  verbose=0,
                                  )(delayed(self._estimator_)(X, y.iloc[:, i]) for i in range(y.shape[1]))
        else :
            print('Fit Progress')
            self.estimators_ = Parallel(n_jobs=self.n_jobs, 
                                  verbose=0,
                                  )(delayed(self._estimator_)(X, y.iloc[:, i]) for i in tqdm(range(y.shape[1])))
        return
    
    def predict(self, X):
        from tqdm.auto import tqdm
        if self.verbose == 0 :
            y_pred = Parallel(n_jobs=self.n_jobs, 
                              verbose=0)(delayed(e.predict)(X) for e in self.estimators_)
        else :
            print('Predict Progress')
            y_pred = Parallel(n_jobs=self.n_jobs, 
                              verbose=0)(delayed(e.predict)(X) for e in tqdm(self.estimators_))
        
        return np.stack(y_pred, axis=1)

In [ ]:
model = ModifiedRegressor(n_jobs=-1, verbose=1)
model.fit(x_linear, y_linear)
y_pred = pd.DataFrame(model.predict(x_linear), index=x.index, columns=y.columns)

In [ ]:
from sklearn.metrics import mean_squared_log_error
y_pred = y_pred.stack(['store_nbr', 'family']).clip(0.)
y_ = y.stack(['store_nbr', 'family']).clip(0.)

y_['pred'] = y_pred.values
print(y_.groupby('family').apply(lambda r : np.sqrt(np.sqrt(mean_squared_log_error(r['sales'], r['pred'])))))
# Looking at error
print('RMSLE : ', np.sqrt(np.sqrt(msle(y_['sales'], y_['pred']))))

In [ ]:
ypred = pd.DataFrame(model.predict(xtest_linear), index = xtest.index, columns = y.columns).clip(0.)
ypred = ypred.stack(['store_nbr', 'family'])

sub = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')
sub['sales'] = ypred.values
sub.to_csv('new_2_submission.csv', index = False) # Submit

In [ ]:
sub.head()

In [ ]:
model_2 = ModifiedRegressor(n_jobs=-1, verbose=1)
model_2.fit(x_xgb, y_xbg)
y_pred = pd.DataFrame(model.predict(x_xgb), index=x.index, columns=y.columns)

y_pred = y_pred.stack(['store_nbr', 'family']).clip(0.)
y_ = y.stack(['store_nbr', 'family']).clip(0.)

y_['pred'] = y_pred.values
print(y_.groupby('family').apply(lambda r : np.sqrt(np.sqrt(mean_squared_log_error(r['sales'], r['pred'])))))
# Looking at error
print('RMSLE : ', np.sqrt(np.sqrt(msle(y_['sales'], y_['pred']))))

ypred = pd.DataFrame(model.predict(xtest_xgb), index = xtest.index, columns = y.columns).clip(0.)
ypred = ypred.stack(['store_nbr', 'family'])

sub = pd.read_csv('../input/store-sales-time-series-forecasting/sample_submission.csv')
sub['sales'] = ypred.values
sub.to_csv('new_xgb_blending_submission.csv', index = False) # Submit

## Arima

In [ ]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

df_train['date'] = pd.to_datetime(df_train.date,format='%Y-%m-%d')
df_train.index = df_train['date']

df_test['date'] = pd.to_datetime(df_test.date,format='%Y-%m-%d')
df_test.index = df_test['date']

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA

Family_list=df_train.family.unique()
store_nbr_list=df_train.store_nbr.unique()

Flag=1
for s in range(len(store_nbr_list)):
  for f in range(len(Family_list)):
    training=df_train[df_train['store_nbr']==store_nbr_list[s]]
    training=training[training['family']==Family_list[f]]
    training_ip=training.drop(['id','date','store_nbr','family','onpromotion'],axis=1)
    model = SARIMAX(training_ip, order = (3,1,1),seasonal_order =(3,1,1,12),simple_differencing=True)
    model = model.fit()
    testing=df_test[df_test['family']==Family_list[f]]
    testing=testing[testing['store_nbr']==store_nbr_list[s]]
    predictions = model.predict(len(training_ip),len(training_ip)+len(testing)+1,typ = 'levels')
    Predictions=predictions[-17:-1]
    forecast = pd.DataFrame(Predictions.values,index = testing.id,columns=['sales'])
    forecast.reset_index(inplace=True)
    if Flag==1:
      Final_op=forecast
    else:
      Final_op = pd.concat([Final_op,forecast])
    Flag+=1
    del training,training_ip,model,testing,Predictions,forecast
    Final_op.to_csv("SARIMA31131112.csv", index=False)

  
Flag=1
for s in range(len(store_nbr_list)):
  for f in range(len(Family_list)):
    training=train[train['store_nbr']==store_nbr_list[s]]
    training=training[training['family']==Family_list[f]]
    training_ip=training.drop(['id','date','store_nbr','family','onpromotion'],axis=1)
    model = ARIMA(training_ip, order=(3,1,1))
    model = model.fit()
    testing=test[test['family']==Family_list[f]]
    testing=testing[testing['store_nbr']==store_nbr_list[s]]
    predictions = model.predict(len(training_ip),len(training_ip)+len(testing)+1,typ = 'levels')
    Predictions=predictions[-17:-1]
    forecast = pd.DataFrame(Predictions.values,index = testing.id,columns=['sales'])
    forecast.reset_index(inplace=True)
    if Flag==1:
      Final_op=forecast
    else:
      Final_op = pd.concat([Final_op,forecast])
    Flag+=1
    del training,training_ip,model,testing,Predictions,forecast
    Final_op.to_csv("ARIMA311.csv", index=False)